In [1]:
!nvidia-smi

Sat Oct 21 13:59:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:01:00.0 Off |                  Off |
| 30%   50C    P8    32W / 300W |      3MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:05:00.0 Off |                  Off |
| 59%   

In [2]:
import os
import torch

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
# !pip install -q bitsandbytes
# !pip install -q transformers 
# !pip install -q peft
# !pip install -q accelerate
# !pip install -q datasets

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0}, pad_token_id=tokenizer.eos_token_id)
model = PeftModel.from_pretrained(model, peft_model_id).to(device)

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=5120, out_features=15360, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features

In [73]:
question = "간단한 문장을 제시하라."
word = "자전거"
prefix_prompt = f'### 입력: {question}\n\n### 답변:'


outputs = []
for idx in range(len(word)):
    char = word[idx]
    
    # encode context the generation is conditioned on
    if len(outputs)==0:
        input_sentence = f'{prefix_prompt}{" ".join(outputs).strip()} {char}'
    else:
        input_sentence = f'{prefix_prompt} {" ".join(outputs).strip()} {char}'
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt').to(device)

    
    if idx == 0:
        beam_output = model.generate(
            input_ids = input_ids, 
            max_length=len(input_ids[0]) + 10,
            early_stopping=True,
            do_sample=True, # 샘플링 전략 사용
            temperature=1.25,
        )
    
        
    elif idx != len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            # max_length=len(input_ids[0]) + 10,
            early_stopping=True,
            # num_beams=10, 
            # no_repeat_ngram_size=2, 
            do_sample=True, # 샘플링 전략 사용
            temperature=0.5,
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )
    
    elif idx == len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            # max_length=len(input_ids[0]) + 10,
            early_stopping=True,
            # num_beams=10, 
            # no_repeat_ngram_size=2, 
            do_sample=False, # 샘플링 전략 사용
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            
            
            # input_ids = input_ids, 
            # max_length=len(input_ids[0]) + 50,
            # early_stopping=True,
            # num_beams=10, 
            no_repeat_ngram_size=2, 
            # do_sample=True, # 샘플링 전략 사용
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )

    output = tokenizer.decode(beam_output[0], skip_special_tokens=True)[len(input_sentence)-1:].split('#')[0].split('. ')[0].strip() # .split('\n')[0]
    outputs.append(output)

    print(f"{char} : {outputs[idx]}")

자 : 자, 이제 내가 생각하는 그는,
전 : 전세계적으로 유명한, 그의 이름은,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
거 : 거시기다.


In [123]:
word = "인하대"
question = f"{word}에 대한 재미있고 간단한 문장을 말씀드리겠습니다."
prefix_prompt = f'{question} '


outputs = []
for idx in range(len(word)):
    char = word[idx]
    
    # encode context the generation is conditioned on
    if len(outputs)==0:
        input_sentence = f'{prefix_prompt}{" ".join(outputs).strip()} {char}'
    else:
        input_sentence = f'{prefix_prompt} {" ".join(outputs).strip()} {char}'
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt').to(device)

    
    if idx == 0:
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            do_sample=True, # 샘플링 전략 사용
            temperature=0.5,
        )
    
        
    elif idx != len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            max_new_tokens = 100,
            early_stopping=True,
            no_repeat_ngram_size=2, 
            do_sample=False, # 샘플링 전략 사용
            temperature=0.25,
            # top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
        )
    
    elif idx == len(word)-1:
        # set no_repeat_ngram_size to 2
        beam_output = model.generate(
            input_ids = input_ids, 
            # max_new_tokens = 100,
            max_length=len(input_ids[0]) + 100,
            early_stopping=True,
            num_beams=10, 
            no_repeat_ngram_size=2, 
            do_sample=True, # 샘플링 전략 사용
            temperature=1,
            top_p=0.90, # 누적 확률이 95%인 후보집합에서만 생성
            # top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
            
        )

        
    output = tokenizer.decode(beam_output[0], skip_special_tokens=True)[len(input_sentence)-1:].split('\n')[0].split('#')[0]
    if '.' in output:
        output = output.split('.')[0] + '.'
    if ',' in output:
        output = output.split(',')[0] + ','

        
    outputs.append(output)

    print(f"{char} : {outputs[idx]}")

인 : 인서울대학이라는 말이 있습니다.
하 : 하물며 인하대는 인인천대학입니다.
대 : 대기업에 취업하는 학생들의 출신대학을 보면 인하대가 아주 높은 위치에 있다는 것을 확인하실 수 있으실 겁니다.
